In [1]:
import numpy as np # linear algebra
import matplotlib.pylab as plt
from random import shuffle
from keras.preprocessing import image
import os
import zipfile
import random
import PIL #import Image
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential

hight = 128
width = 128
CHANNELS = 3

from keras.applications import VGG16
from keras.applications import *
from keras import Model
from keras.layers import Input, Dense


Using Theano backend.


In [2]:
#insert all images to correct arrays
cats=[]
dogs=[]
jackels=[]
sheeps=[]
for file in os.listdir('C:/Users/Dar/Desktop/untitled3/train/images/cat'):
    if file.endswith(".jpg"):
        cats.append(file)
for file in os.listdir('C:/Users/Dar/Desktop/untitled3/train/images/dog'):
    if file.endswith(".jpg"):
        dogs.append(file)
for file in os.listdir('C:/Users/Dar/Desktop/untitled3/train/images/jackel'):
    if file.endswith(".jpg"):
        jackels.append(file)
for file in os.listdir('C:/Users/Dar/Desktop/untitled3/train/images/sheep'):
    if file.endswith(".jpg"):
        sheeps.append(file)
total_len=len(cats)+len(dogs)+len(jackels)+len(sheeps)
#print(total_len)

In [3]:
#initializing a matrix of zeros with the required input structure
data = np.zeros(shape=(total_len,hight,width,CHANNELS),dtype=np.uint8)
#print(data)
#initializing a matrix of zeros with the required output structure
labels = np.zeros((total_len,4))
#print(labels)

In [4]:

for i,img_path in enumerate(cats):
    img = image.load_img('C:/Users/Dar/Desktop/untitled3/train/images/cat/'+img_path, target_size=(hight,width))
    data[i]=image.img_to_array(img)
    labels[i,0] = 1
print("cats")
for i,img_path in enumerate(dogs):
    img = image.load_img('C:/Users/Dar/Desktop/untitled3/train/images/dog/'+img_path, target_size=(hight,width))
    data[i+len(cats)]=image.img_to_array(img)
    labels[i+len(cats),1] = 1
print("dogs")   
for i,img_path in enumerate(jackels):
    img = image.load_img('C:/Users/Dar/Desktop/untitled3/train/images/jackel/'+img_path, target_size=(hight,width))
    data[i+len(cats)+len(dogs)]=image.img_to_array(img)
    labels[i+len(cats)+len(dogs),2] = 1
print("j")
for i,img_path in enumerate(sheeps):
    img = image.load_img('C:/Users/Dar/Desktop/untitled3/train/images/sheep/'+img_path, target_size=(hight,width))
    data[i+len(cats)+len(dogs)+len(jackels)]=image.img_to_array(img)
    labels[i+len(cats)+len(dogs)+len(jackels),3] = 1    
print("s")

cats
dogs
j
s


In [5]:
#plt.imshow(data[0])
#The final preprocessing step for the input data is to convert our data type to float32 and normalize our data values to the range [0, 1].
data = data.astype('float32')
data /= 255

#randomly divide the data to train and test sets
indices = np.arange(len(data))
random.shuffle(indices)
train_x = data[indices[:int(len(indices)*0.8)]]
train_y = labels[indices[:int(len(indices)*0.8)]]
test_x = data[indices[int(len(indices)*0.8):]]
test_y = labels[indices[int(len(indices)*0.8):]]
print(len(test_x))
print(len(train_x))

371
1480


In [6]:
#to calculate the classification accuracy, precision and recall for each class, we divided the test set according to classes. 
where_are_cats=[]
where_are_dogs=[]
where_are_jackals=[]
where_are_sheeps=[]

for i in range(len(test_y)):
    if test_y[i][0]==1:
        where_are_cats.append(i)
    if test_y[i][1]==1:
        where_are_dogs.append(i)
    if test_y[i][2]==1:
        where_are_jackals.append(i)
    if test_y[i][3]==1:
        where_are_sheeps.append(i)

print(len(where_are_cats))
print(len(where_are_dogs))
print(len(where_are_jackals))
print(len(where_are_sheeps))

test_y_cats = np.zeros((len(where_are_cats),4))
test_x_cats= np.zeros(shape=(len(where_are_cats),hight,width,CHANNELS),dtype=np.float32)
test_y_dogs = np.zeros((len(where_are_dogs),4))
test_x_dogs= np.zeros(shape=(len(where_are_dogs),hight,width,CHANNELS),dtype=np.float32)
test_y_jackals = np.zeros((len(where_are_jackals),4))
test_x_jackals= np.zeros(shape=(len(where_are_jackals),hight,width,CHANNELS),dtype=np.float32)
test_y_sheeps = np.zeros((len(where_are_sheeps),4))
test_x_sheeps= np.zeros(shape=(len(where_are_sheeps),hight,width,CHANNELS),dtype=np.float32)

for i in range(len(where_are_cats)):
    test_x_cats[i]=test_x[where_are_cats[i]]
    test_y_cats[i]=test_y[where_are_cats[i]]
for i in range(len(where_are_dogs)):
    test_x_dogs[i]=test_x[where_are_dogs[i]]
    test_y_dogs[i]=test_y[where_are_dogs[i]]
for i in range(len(where_are_jackals)):
    test_x_jackals[i]=test_x[where_are_jackals[i]]
    test_y_jackals[i]=test_y[where_are_jackals[i]]
for i in range(len(where_are_sheeps)):
    test_x_sheeps[i]=test_x[where_are_sheeps[i]]
    test_y_sheeps[i]=test_y[where_are_sheeps[i]]

#print(test_y_cats)
#print(test_x_cats)


115
103
45
108


In [ ]:
#create the classification model
model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(hight,width,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_x, train_y,batch_size=32, nb_epoch=14, verbose=1)
score1=model.evaluate(train_x,train_y, verbose=0)

#print scores
print("train score:")
print(score1)
score = model.evaluate(test_x, test_y, verbose=0)
print("total test score:")
print(score)
score_cat = model.evaluate(test_x_cats, test_y_cats, verbose=0)
print("cat score:")
print(score_cat)
score_dog = model.evaluate(test_x_dogs, test_y_dogs, verbose=0)
print("dog score:")
print(score_dog)
score_jackal = model.evaluate(test_x_jackals, test_y_jackals, verbose=0)
print("jackel score:")
print(score_jackal)
score_sheep = model.evaluate(test_x_sheeps, test_y_sheeps, verbose=0)
print("sheep score:")
print(score_sheep)

C:\Users\Dar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128,...)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Dar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  """
C:\Users\Dar\Anaconda3\lib\site-packages\keras\models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/14
1480/1480 [==============================] - 82s 56ms/step - loss: 1.2947 - acc: 0.4034
Epoch 2/14
 544/1480 [==========>...................] - ETA: 59s - loss: 1.0815 - acc: 0.5018 

In [15]:
#calculate precision and recall for each class
from sklearn.metrics import classification_report
y_pred = model.predict(test_x)
predicted = []
labels={0:"Cats", 1:"Dogs", 2:"Jackels", 3:"Sheeps"}
for j in labels.keys():
    tp=0; fn=0; fp=0; tn=0
    for i in range(len(y_pred)):
        really_is=test_y[i].tolist().index(max(test_y[i]))
        classified_as=y_pred[i].tolist().index(max(y_pred[i]))
        if really_is==j and classified_as==j:
            tp+=1
        elif really_is!=j and classified_as!=j:
            tn+=1
        elif really_is==j and classified_as!=j:
            fn+=1
        else:
            fp+=1
    print(labels[j])
    print("Recall: "+str(round(tp/(tp+fn),3)))
    print("Precision: "+str(round(tp/(tp+fp),3)))
    print("\n")

Cats
Recall: 0.6
Precision: 0.63


Dogs
Recall: 0.563
Precision: 0.713


Jackels
Recall: 0.796
Precision: 0.542


Sheeps
Recall: 0.878
Precision: 0.819




In [16]:
from keras.models import load_model
model.save("model.h5")